In [ ]:
import re
import string
import pandas as pd
import numpy as np

In [ ]:
tweets = pd.read_csv('final.csv').iloc[:,1:]
tweet = tweets['Tweets']

In [ ]:
stop_words = ['a', 'an', 'the']
punct_words = list(string.punctuation)
stop = stop_words + punct_words
stop = list(set(stop))

In [ ]:
def process_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(f'[{string.punctuation}]', '', text)
    words = text.split()
    words = [word for word in words if word not in stop]
    return ' '.join(words)

In [ ]:
tweet0 = tweet[:10]
processed_tweet0 = tweet0.apply(process_text)

In [ ]:
def phase_1(lines):
    n = len(lines)
    lines_thresh_bool = np.ones((n * n,), dtype=np.bool)
    for i in range(n):
        for j in range(i + 1, n):
            line1 = lines[i]
            n1 = len(line1)
            line2 = lines[j]
            n2 = len(line2)
            n_max = max(n1, n2)
            n_min = min(n1, n2)
            count_true = 0
            for k in range(n_min):
                if line1[k] == line2[k]:
                    count_true += 1
            thresh_true = round(count_true / n_max, 2)
            thresh_false = 1 - thresh_true
            if (thresh_true <= 0.25) or (0.75 <= thresh_false):
                lines_thresh_bool[(n * i) + j] = False
                lines_thresh_bool[i + (n * j)] = False
    return lines_thresh_bool.reshape(n, n)

In [ ]:
sample_lines = ['This is a test', 'Another test line', 'Something different']
result = phase_1(sample_lines)